In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.nasnet import NASNetMobile,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortex"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = NASNetMobile(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1056)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1056)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   36.7s


[CV] ........................... C=0.01, score=0.869094, total=  37.0s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.875490, total=  39.9s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.890411, total=  40.6s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.890208, total=  40.0s
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.882937, total=  41.9s
[CV] ........................... C=0.01, score=0.890748, total=  42.5s
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.881474, total=  41.5s
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.2min


[CV] ............................ C=0.1, score=0.912957, total= 2.1min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.901961, total= 2.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.915851, total= 2.4min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.908730, total= 2.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.906250, total= 2.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.917740, total= 2.6min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.895669, total= 2.7min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.908465, total= 4.6min
[CV] ................... C=1000000000.0, score=0.899110, total= 4.5min
[CV] ................... C=1000000000.0, score=0.893954, total= 4.5min
[CV] .................... C=100000000.0, score=0.871514, total= 5.3min
[CV] .................... C=100000000.0, score=0.893638, total= 5.4min
[CV] ................... C=1000000000.0, score=0.880906, total= 5.4min
[CV] ................... C=1000000000.0, score=0.889881, total= 4.4min
[CV] ................... C=1000000000.0, score=0.892644, total= 4.1min
[CV] ................... C=1000000000.0, score=0.895418, total= 4.2min
[CV] .................. C=10000000000.0, score=0.882874, total= 4.0min
[CV] .................. C=10000000000.0, score=0.896078, total= 4.1min
[CV] .................. C=10000000000.0, score=0.890873, total= 3.7min
[CV] .................. C=10000000000.0, score=0.898910, total= 3.7min
[CV] .................. C=10000000000.0, score=0.905045, total= 3.9min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 21.1min remaining:   50.7s


[CV] .................. C=10000000000.0, score=0.876740, total= 3.9min
[CV] .................. C=10000000000.0, score=0.902559, total= 4.2min
[CV] .................. C=10000000000.0, score=0.904297, total= 4.9min
[CV] .................. C=10000000000.0, score=0.892430, total= 4.0min
[CV] .................. C=10000000000.0, score=0.898239, total= 4.9min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 21.8min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 39.04641774, 137.80572698, 185.38115735, 191.57466936,
        189.70631597, 227.99740334, 268.1298651 , 295.04629209,
        288.18662908, 294.91653965, 285.25620763, 270.8144346 ,
        248.28031063]),
 'mean_score_time': array([0.05331624, 0.03211222, 0.04201159, 0.02482457, 0.02774379,
        0.04030895, 0.03949339, 0.03439453, 0.04471178, 0.04135365,
        0.04054286, 0.03423409, 0.02041728]),
 'mean_test_score': array([0.88417522, 0.90805051, 0.90607735, 0.89680347, 0.89443567,
        0.89542226, 0.89492897, 0.89443567, 0.89581689, 0.89542226,
        0.89265983, 0.89453433, 0.89483031]),
 'mean_train_score': array([0.91029763, 0.96229053, 0.98954193, 0.99211896, 0.98975699,
        0.9929303 , 0.99213007, 0.99163331, 0.99060572, 0.99193114,
        0.98822134, 0.99221699, 0.99147243]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.88418, std: 0.00688, params: {'C': 0.01},
 mean: 0.90805, std: 0.00695, params: {'C': 0.1},
 mean: 0.90608, std: 0.00843, params: {'C': 1.0},
 mean: 0.89680, std: 0.00852, params: {'C': 10.0},
 mean: 0.89444, std: 0.00715, params: {'C': 100.0},
 mean: 0.89542, std: 0.00911, params: {'C': 1000.0},
 mean: 0.89493, std: 0.00986, params: {'C': 10000.0},
 mean: 0.89444, std: 0.01142, params: {'C': 100000.0},
 mean: 0.89582, std: 0.01188, params: {'C': 1000000.0},
 mean: 0.89542, std: 0.01184, params: {'C': 10000000.0},
 mean: 0.89266, std: 0.01005, params: {'C': 100000000.0},
 mean: 0.89453, std: 0.00725, params: {'C': 1000000000.0},
 mean: 0.89483, std: 0.00880, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.1} - score: 0.9081
